# GROUPING

After preparing the data for a more exhaustive analysis, the last step is to separate the different countries into group with common characteristics. This is necessary as an aggregate study of all countries would mean erasing the differences between them and missing important conclusions, making our study too shallow.

Grouping them by similar characteristics will simplify our analysis, all while reducing the data loss and stressing common patterns among countries in the same group.

In [1]:
import os
import numpy as np
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from Project.Utils.aggregate import aggregate


column_country = 'Country'
column_year = 'Year'
column_region = 'Region'

read_path = os.getcwd()
region_url = read_path + '/Databases/AuxiliarData/world-regions.csv'
df_url = read_path + '/Output/SilverDataframe.csv'

write_path = os.getcwd() + '/Output/'
country_folder = write_path + '/Country/'
region_folder = write_path + '/Region/'
aggregate_folder = region_folder + '/Aggregate/'



## CREATING DIRECTORIES
If the destination folders do not exist, create them.

In [2]:
if not os.path.exists(country_folder):
            os.makedirs(country_folder)
if not os.path.exists(region_folder):
            os.makedirs(region_folder)
if not os.path.exists(aggregate_folder):
            os.makedirs(aggregate_folder)

### READ THE FULL DATAFRAME AND THE REGION DATAFRAME

In [3]:
region_df = pd.read_csv(region_url)
df = pd.read_csv(df_url, index_col = ['Country', 'Year'])

print(region_df)
region_df.drop(columns=['Code', 'Year'], inplace = True)
region_df.rename(columns = {'World Region according to the World Bank': column_region, 'Entity': column_country}, inplace = True)
#region_df.set_index(['Country', 'Region'], inplace = True)
print(region_df)

             Entity Code  Year World Region according to the World Bank
0       Afghanistan  AFG  2017                               South Asia
1           Albania  ALB  2017                  Europe and Central Asia
2           Algeria  DZA  2017             Middle East and North Africa
3    American Samoa  ASM  2017                    East Asia and Pacific
4           Andorra  AND  2017                  Europe and Central Asia
..              ...  ...   ...                                      ...
212       Venezuela  VEN  2017              Latin America and Caribbean
213         Vietnam  VNM  2017                    East Asia and Pacific
214           Yemen  YEM  2017             Middle East and North Africa
215          Zambia  ZMB  2017                       Sub-Saharan Africa
216        Zimbabwe  ZWE  2017                       Sub-Saharan Africa

[217 rows x 4 columns]
            Country                        Region
0       Afghanistan                    South Asia
1           

## ADD THE PROPERTY 'REGION'
By merging both DataFrames, we will obtain a DataFrame similar to the original one with a new field for the index: `region`

In [4]:
print(df)
region_df.set_index(['Country', 'Region'], inplace = True)
df = pd.merge(df, region_df, how = 'inner', left_index = True, right_index = True)
print(df)

                  AgriShareGDP  CreditToAgriFishForest  EmploymentRural  \
Country     Year                                                          
Afghanistan 2000      54.06300                     NaN              NaN   
            2001      54.06300                     NaN              NaN   
            2002      45.13440                     NaN              NaN   
            2003      41.90340                     NaN              NaN   
            2004      35.61280                     NaN              NaN   
...                        ...                     ...              ...   
Zimbabwe    2016       7.87399                     NaN              NaN   
            2017       8.34095                     NaN              NaN   
            2018       8.30469                     NaN              NaN   
            2019       8.17322                     NaN              NaN   
            2020      10.93630                     NaN              NaN   

                        

## EXPORT THE SEGREGATED DATAFRAMES
We will export the new DataFrame with the regions and two new kinds of DataFrame: one for every different country, and another for every region.

In [5]:
df.to_csv(write_path + '/GoldDataframe.csv')

country_list = set(df.index.get_level_values(0))
region_list = set(df.index.get_level_values(2))

for country in country_list:
    df.loc[df.index.get_level_values(0) == country].to_csv(country_folder + country + '.csv')

for region in region_list:
    df.loc[df.index.get_level_values(2) == region].to_csv(region_folder + region + '.csv')

# REGION AGGREGATES
In order to create an aggregated DataFrame with entries for the regions we are studying, we will have to estimate the values of its indicators: the summatory for absolute ones, and a weighted average based on population for the relative ones. The weight of the mean could also be established as the GDP of each country, or any other figure we consider representative enough of the country's importance in the calculus.

In [6]:
#Define which indicators should be not a weighted
abs_indicators = ['GDP', 'Population', 'TotalAgri']

#Wether aggregate from the regions aggregate (faster) or from the country raw data (slower but more precise)
fast_aggregate = True

aggregate_df = pd.DataFrame()

for region in region_list:
    #Create sub-DataFrame with only the countries for each region
    region_df = df.loc[df.index.get_level_values(column_region) == region]
    #Aggregate by the countries of said region
    aggregated_region_df = aggregate(region_df, aggregate_by = column_country, for_index = column_year, new_group_col_name = column_region, group_name = region, abs_indicators = abs_indicators)
    #Add the aggregated DataFrame to the DataFrame per aggregated regions
    aggregate_df = pd.concat([aggregate_df, aggregated_region_df])

#Obtain an aggregate for the world
if fast_aggregate:
    aggregate_df_world = aggregate(aggregate_df, column_region, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)
else:
    aggregate_df_world = aggregate(df, column_country, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)

#Export the aggregated DataFrame to a .csv file
aggregate_df.to_csv(write_path + 'AggregatedRegion_DataFrame.csv')

aggregate_df_world.to_csv(write_path + 'AggregatedWorld_DataFrame.csv')

In [7]:
import datetime

#Set True if you want to analyse the time for both methods. By default is False to save that time.
test_time = False

if test_time:
    #Count time for the fast method: aggregating from the aggregated regions.
    st_time = datetime.now()
    fast_aggregate_df = aggregate(aggregate_df, column_region, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)
    f_time = datetime.now()
    fast_time = f_time - st_time
    print('Fast aggregation time: ' + fast_time.total_seconds())

    #Count time for the most precise method: aggregating from the raw data per country.
    st_time = datetime.now()
    precise_aggregate_df = aggregate(df, column_country, column_year, new_group_col_name = column_region, group_name = 'World',abs_indicators = abs_indicators)
    f_time = datetime.now()
    precise_time = f_time - st_time
    print('Precisse aggregation time: ' + precise_time.total_seconds())

    #Print the results to compare.
    print(fast_aggregate_df)
    print(precise_aggregate_df)

    #Export the results to .csv files if a more detailed analysis is needed.
    fast_aggregate_df.to_csv(region_folder + 'World_Fast.csv')
    precise_aggregate_df.to_csv(region_folder + 'World.csv')
else:
    print('No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again')

No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again


### Variables that can be changed
The p-value can be changed to a desired value. For the default value is recommnended 0.05. If you desire a higher confidence level lower this variable. This variable reffers directly to significance level, but for porpouses of clarification is set to this name:

We do the %store in order to pass this variable to other notebooks.

In [8]:
PVALUE_VAR = 0.05

%store PVALUE_VAR

Stored 'PVALUE_VAR' (float)


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 
This codeblock also calculates the p-value of Spearman and Pearson, if the value > PVALUE_VAR the correlation will be deleted due to not having statiscal significance.

Later on is concatenated and generates the following result:

In [9]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Belarus,-0.740688,NaN,NaN,0.969914,NaN,0.714361,-0.572392,-0.898360,0.758016,-0.919550,...,0.753786,-0.809223,NaN,NaN,NaN,-0.801186,NaN,NaN,NaN,-0.932844
Portugal,-0.937049,NaN,-0.519219,0.941841,NaN,-0.684102,NaN,NaN,0.660173,-0.715156,...,-0.633610,-0.680719,-0.718688,NaN,-0.511585,-0.676486,NaN,-0.624523,NaN,NaN
French Polynesia,-0.833535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.860169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burundi,NaN,NaN,NaN,0.829954,-0.837539,-0.842824,-0.970891,NaN,0.978385,-0.946126,...,NaN,-0.987871,-0.921282,-0.821986,0.655949,-0.617099,0.921316,NaN,NaN,0.988052
Lithuania,-0.835335,NaN,-0.788530,0.912241,0.706347,0.574701,0.867250,-0.837935,0.746119,-0.812293,...,NaN,-0.633390,NaN,NaN,NaN,-0.447208,-0.940626,NaN,0.637510,-0.922340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iraq,-0.793279,NaN,NaN,0.824084,NaN,-0.781195,-0.788318,NaN,0.645700,-0.650486,...,NaN,-0.806012,NaN,NaN,0.476742,0.533647,NaN,NaN,NaN,0.852593
Mauritania,-0.696230,NaN,NaN,NaN,-0.946624,-0.886282,-0.938461,NaN,0.935888,-0.904177,...,NaN,-0.955581,NaN,NaN,0.896664,0.896081,NaN,NaN,NaN,0.938437
Sweden,-0.693931,NaN,-0.637063,0.870518,-0.875553,0.732351,-0.851310,NaN,0.892014,-0.435390,...,0.817130,-0.723628,-0.477288,NaN,0.819141,-0.878838,0.661258,-0.869114,-0.810861,0.811345
Congo,-0.636275,NaN,NaN,0.804072,-0.725357,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.697859


## Cleaned GoldDataframe 
Before the correlation dataframe was generated and it detected all the correlations with a high p-value. To perform all the following analysis a clean GoldDataframe is needed. It simply detects if the correlation for the indicator and country is Nan and procedes to delete the whole column indicator for the country.

In [10]:

df = df.copy()
for country in countries:
    for ind in corr_df.columns:
        if np.isnan(corr_df[ind][country]):
            df.loc[df[col_country] == country, [ind]] = np.NaN

df.set_index([col_country, col_region, col_year]).to_csv(write_path + 'GoldDataframe_significant.csv')
df.set_index([col_country, col_region, col_year])

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  TotalAgri  \
Country     Region             Year                                             
Afghanistan South Asia         2000              NaN   3342.034168        NaN   
                               2001              NaN   3598.470576        NaN   
                               2002              NaN   4141.523943        NaN   
                               2003              NaN   4729.042179        NaN   
                               2004              NaN   5388.482107        NaN   
...                                              ...           ...        ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073        NaN   
                               2017              NaN  22040.902301        NaN   
                               2018              NaN  24311.560545        NaN   
                               2019              NaN  21935.075306        NaN   
                               2020              NaN  21786.743637        NaN   

                                     % Soldiers  Birth Rate  Death Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000         NaN      48.021      11.718   
                               2001         NaN      47.505      11.387   
                               2002         NaN      46.901      11.048   
                               2003         NaN      46.231      10.704   
                               2004         NaN      45.507      10.356   
...                                         ...         ...         ...   
Zimbabwe    Sub-Saharan Africa 2016     0.75975         NaN       8.286   
                               2017     0.75072         NaN       8.044   
                               2018     0.73821         NaN       7.883   
                               2019     0.73821         NaN       7.773   
                               2020     0.73821         NaN       7.692   

                                     Homicides  Life Expectancy  ...  \
Country     Region             Year                              ...   
Afghanistan South Asia         2000        NaN           55.841  ...   
                               2001        NaN           56.308  ...   
                               2002        NaN           56.784  ...   
                               2003        NaN           57.271  ...   
                               2004        NaN           57.772  ...   
...                                        ...              ...  ...   
Zimbabwe    Sub-Saharan Africa 2016        NaN           60.294  ...   
                               2017        NaN           60.812  ...   
                               2018        NaN           61.195  ...   
                               2019        NaN           61.490  ...   
                               2020        NaN           61.738  ...   

                                     % Population Growth  % Rural Po

## Saving Correlation Dataframe Pearson

Use the same code as before, but this time save the correlation dataframe.

In [11]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe_significant.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF_pearson.csv')
corr_df

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Belarus,-0.740688,NaN,NaN,0.969914,NaN,0.714361,-0.572392,-0.898360,0.758016,-0.919550,...,0.753786,-0.809223,NaN,NaN,NaN,-0.801186,NaN,NaN,NaN,-0.932844
Portugal,-0.937049,NaN,-0.519219,0.941841,NaN,-0.684102,NaN,NaN,0.660173,-0.715156,...,-0.633610,-0.680719,-0.718688,NaN,-0.511585,-0.676486,NaN,-0.624523,NaN,NaN
French Polynesia,-0.833535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.860169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burundi,NaN,NaN,NaN,0.829954,-0.837539,-0.842824,-0.970891,NaN,0.978385,-0.946126,...,NaN,-0.987871,-0.921282,-0.821986,0.655949,-0.617099,0.921316,NaN,NaN,0.988052
Lithuania,-0.835335,NaN,-0.788530,0.912241,0.706347,0.574701,0.867250,-0.837935,0.746119,-0.812293,...,NaN,-0.633390,NaN,NaN,NaN,-0.447208,-0.940626,NaN,0.637510,-0.922340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iraq,-0.793279,NaN,NaN,0.824084,NaN,-0.781195,-0.788318,NaN,0.645700,-0.650486,...,NaN,-0.806012,NaN,NaN,0.476742,0.533647,NaN,NaN,NaN,0.852593
Mauritania,-0.696230,NaN,NaN,NaN,-0.946624,-0.886282,-0.938461,NaN,0.935888,-0.904177,...,NaN,-0.955581,NaN,NaN,0.896664,0.896081,NaN,NaN,NaN,0.938437
Sweden,-0.693931,NaN,-0.637063,0.870518,-0.875553,0.732351,-0.851310,NaN,0.892014,-0.435390,...,0.817130,-0.723628,-0.477288,NaN,0.819141,-0.878838,0.661258,-0.869114,-0.810861,0.811345
Congo,-0.636275,NaN,NaN,0.804072,-0.725357,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.697859


Saving the correlation dataframe using spearman.

In [12]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe_significant.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr(method='spearman')

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF_spearman.csv')
corr_df

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Belarus,-0.639818,NaN,NaN,0.950649,NaN,0.664499,-0.462694,-0.745124,0.745047,-0.741372,...,0.706494,-0.749351,NaN,NaN,NaN,-0.698375,NaN,NaN,NaN,-0.812745
Portugal,-0.862439,NaN,-0.448635,0.767231,NaN,-0.488107,NaN,NaN,0.533984,-0.563602,...,-0.456437,-0.546164,-0.514919,NaN,-0.468943,-0.554469,NaN,-0.464666,NaN,NaN
French Polynesia,-0.679012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.597403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burundi,NaN,NaN,NaN,0.809091,-0.925879,-0.748052,-0.981818,NaN,0.981818,-0.978507,...,NaN,-0.981818,-0.934373,-0.769981,0.671910,-0.629319,0.907673,NaN,NaN,0.981818
Lithuania,-0.685287,NaN,-0.777922,0.840260,0.524058,0.491378,0.695029,-0.832250,0.744156,-0.816942,...,NaN,-0.654321,NaN,NaN,NaN,-0.614987,-0.850445,NaN,0.658838,-0.910390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Iraq,-0.774277,NaN,NaN,0.871666,NaN,-0.876623,-0.758285,NaN,0.622078,-0.729646,...,NaN,-0.876623,NaN,NaN,0.535381,0.535508,NaN,NaN,NaN,0.876623
Mauritania,-0.601494,NaN,NaN,NaN,-0.944084,-0.949351,-0.949351,NaN,0.949351,-0.943979,...,NaN,-0.949351,NaN,NaN,0.903416,0.813685,NaN,NaN,NaN,0.949351
Sweden,-0.433258,NaN,-0.459740,0.859740,-0.745124,0.526424,-0.814575,NaN,0.828571,-0.493162,...,0.677922,-0.815584,-0.578107,NaN,0.802080,-0.813685,0.653424,-0.648380,-0.461448,0.815584
Congo,-0.635921,NaN,NaN,0.806494,-0.750649,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.716883
